<a href="https://colab.research.google.com/github/jsebastianquiroga/PUJ_NLP_IA/blob/main/proyecto/inferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.9 MB/s eta 0:00:00


In [5]:
pip install -U typing_extensions


In [3]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl si

In [6]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import gdown
import joblib

# Check for GPU and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Download and load the pre-trained k-NN model
gdown.download('https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q', 'knn_model.pkl', quiet=False)
knn_model = joblib.load('knn_model.pkl')

# Download and load the dataset with pre-computed embeddings
gdown.download('https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf', 'merged_df_embeddings.parquet', quiet=False)
merged_df = pd.read_parquet('merged_df_embeddings.parquet')

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Exclude the first index as it will be the book itself
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
import gradio as gr

# Define the interface
def gradio_recommend_books(user_input):
    recommended_books = recommend_books(user_input)
    return recommended_books.to_dict(orient='records')

iface = gr.Interface(
    fn=gradio_recommend_books,
    inputs="text",
    outputs="dataframe",
    title="Book Recommendation System",
    description="Enter your text to get book recommendations based on your interests."
)

iface.launch()


Using device: cuda


Downloading...
From: https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q
To: /content/knn_model.pkl
100%|██████████| 417M/417M [00:02<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf
To: /content/merged_df_embeddings.parquet
100%|██████████| 546M/546M [00:07<00:00, 68.5MB/s]


ImportError: ignored

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import gdown
import joblib
import numpy as np

# Check for GPU and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Download and load the pre-trained k-NN model
gdown.download('https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q', 'knn_model.pkl', quiet=False)
knn_model = joblib.load('knn_model.pkl')

# Download and load the dataset with pre-computed embeddings
gdown.download('https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf', 'merged_df_embeddings.parquet', quiet=False)
merged_df = pd.read_parquet('merged_df_embeddings.parquet')

def recommend_books_1(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])

    # Normalize the review scores to [0, 1]
    max_score = merged_df['review/score'].max()
    min_score = merged_df['review/score'].min()
    merged_df['normalized_score'] = (merged_df['review/score'] - min_score) / (max_score - min_score)

    # Adjust distances based on review scores
    for i, index in enumerate(indices[0]):
        distances[0][i] *= (1 - merged_df.iloc[index]['normalized_score'])

    # Now, sort the books based on adjusted distances
    sorted_indices = np.argsort(distances[0])

    # Get top 5 books based on adjusted distances
    recommended_books = merged_df.iloc[sorted_indices[1:6]]  # Exclude the first index as it will be the book itself

    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
recommend_books_1('I want to read something fantasy and love')

Using device: cuda


Downloading...
From: https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q
To: /content/knn_model.pkl
100%|██████████| 417M/417M [00:04<00:00, 101MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf
To: /content/merged_df_embeddings.parquet
100%|██████████| 546M/546M [00:03<00:00, 159MB/s]


,Title,authors,publishedDate
5,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],1996
8,Saint Hyacinth of Poland,['Mary Fabyan Windeatt'],2009-01-01
9,Rising Sons and Daughters: Life Among Japan's ...,['Steven Wardell'],1995
2,Wonderful Worship in Smaller Churches,['David R. Ray'],2000
3,Whispers of the Wicked Saints,['Veronica Haddon'],2005-02


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import gdown
import joblib
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Check for GPU and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Download and load the pre-trained k-NN model
gdown.download('https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q', 'knn_model.pkl', quiet=False)
knn_model = joblib.load('knn_model.pkl')

# Download and load the dataset with pre-computed embeddings
gdown.download('https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf', 'merged_df_embeddings.parquet', quiet=False)
merged_df = pd.read_parquet('merged_df_embeddings.parquet')

# Function for cosine similarity based recommendations
def recommend_based_on_text(new_text):
    # Generate embeddings for the new text
    new_embedding = generate_embeddings(new_text)

    # Calculate similarity between new embedding and all book embeddings
    similarities = cosine_similarity([new_embedding], merged_df['combined_embedding'].tolist())

    # Weight by rating
    # Assuming 'review/score' is normalized between [0, 1]
    weighted_scores = similarities * merged_df['review/score'].values

    # Recommend
    recommended_indices = weighted_scores.argsort()[0][::-1][:5]  # Top 5 recommendations
    recommended_books = merged_df.iloc[recommended_indices]

    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
recommend_based_on_text("I want to read something fantasy and love")


Using device: cuda


Downloading...
From: https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q
To: /content/knn_model.pkl
100%|██████████| 417M/417M [00:04<00:00, 87.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf
To: /content/merged_df_embeddings.parquet
100%|██████████| 546M/546M [00:02<00:00, 243MB/s]


,Title,authors,publishedDate
173491,Curious George's Dream (Turtleback School & Li...,"['H. A. Rey', 'Erica Zappy']",2015-06-15
128199,The Canary Chronicles: The Road to Maura,['Anne Smith Burton'],2004-10
103584,Chamario: Libro De Rimas Para Ninos (Rimas Y A...,['Parragon'],2015-04-01
55116,Rimas/ Rhymes (Clasicos / Classics) (Spanish E...,['Parragon'],2015-04-01
153173,Stuart Little (E.B. White) (Novel units),"['Novel Units', 'Anne Troy']",2000
